## Generating joint load-solar scenarios using PGscen ##

The final and most complex model included in PGscen models load demand and solar generator output jointly in order to capture the correlation between the two caused by weather patterns. This model can be invoked using the ```pgscen-load-solar``` command.

For this model, we need to get both load and solar datasets.

In [ ]:
from pathlib import Path
import pandas as pd
from pgscen.command_line import (
    load_load_data, load_solar_data,
    split_actuals_hist_future, split_forecasts_hist_future
    )

import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import matplotlib.dates as mdates
%matplotlib inline
plt.rcParams['figure.figsize'] = [19, 11]
from IPython.display import display

start_date = '2018-07-17'
cur_path = Path("day-ahead_solar.ipynb").parent.resolve()
data_dir = Path(cur_path, '..', "data").resolve()

scenario_count = 1000
scen_start_time = pd.to_datetime(' '.join([start_date, "06:00:00"]), utc=True)
scen_timesteps = pd.date_range(start=scen_start_time, periods=24, freq='H')

load_zone_actual_df, load_zone_forecast_df = load_load_data()
(solar_site_actual_df, solar_site_forecast_df,
     solar_meta_df) = load_solar_data()

(load_zone_actual_hists,
    load_zone_actual_futures) = split_actuals_hist_future(
            load_zone_actual_df, scen_timesteps)
(load_zone_forecast_hists,
     load_zone_forecast_futures) = split_forecasts_hist_future(
            load_zone_forecast_df, scen_timesteps)

(solar_site_actual_hists,
     solar_site_actual_futures) = split_actuals_hist_future(
            solar_site_actual_df, scen_timesteps)
(solar_site_forecast_hists,
     solar_site_forecast_futures) = split_forecasts_hist_future(
            solar_site_forecast_df, scen_timesteps)

fig, ((load_hist_ax, load_future_ax),
      (solar_hist_ax, solar_future_ax)) = plt.subplots(
    figsize=(17, 11), nrows=2, ncols=2)

title_args = dict(weight='semibold', size=19)
actual_clr, fcst_clr = "#430093", "#D9C800"
load_plt_asset = 'Coast'
solar_plt_asset = 'Angelo Solar'

load_hist_ax.set_title("Load History", **title_args)
load_hist_ax.plot(load_zone_actual_hists[load_plt_asset][-250:], c=actual_clr)
load_hist_ax.plot(load_zone_forecast_hists['Forecast_time'][-250:],
                  load_zone_forecast_hists[load_plt_asset][-250:],
                  c=fcst_clr)
load_hist_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

load_future_ax.set_title("Load Future", **title_args)
load_future_ax.plot(load_zone_actual_futures[load_plt_asset][:250], c=actual_clr)
load_future_ax.plot(load_zone_forecast_futures['Forecast_time'][:250],
                    load_zone_forecast_futures[load_plt_asset][:250], c=fcst_clr)
load_future_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

solar_hist_ax.set_title("Solar History", **title_args)
solar_hist_ax.plot(solar_site_actual_hists[solar_plt_asset][-250:], c=actual_clr)
solar_hist_ax.plot(solar_site_forecast_hists['Forecast_time'][-250:],
                   solar_site_forecast_hists[solar_plt_asset][-250:],
                   c=fcst_clr)
solar_hist_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

solar_future_ax.set_title("Solar Future", **title_args)
solar_future_ax.plot(solar_site_actual_futures[solar_plt_asset][:250], c=actual_clr)
solar_future_ax.plot(solar_site_forecast_futures['Forecast_time'][:250],
                     solar_site_forecast_futures[solar_plt_asset][:250], c=fcst_clr)
solar_future_ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))

for ax in [load_hist_ax, load_future_ax, solar_hist_ax, solar_future_ax]:
    ax.tick_params(which='both', labelsize=13)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    ax.grid(linewidth=0.9, alpha=0.41)
    ax.axhline(0, lw=1.3, c='black', alpha=1)
    ax.set_ylim((0, ax.get_ylim()[1]))

lgnd_ptchs = [Patch(color=actual_clr, alpha=0.53, label="Actuals"),
              Patch(color=fcst_clr, alpha=0.53, label="Forecasts")]

_ = fig.legend(handles=lgnd_ptchs, frameon=False, fontsize=21, ncol=2, loc=8,
               bbox_to_anchor=(0.5, 0), handletextpad=0.7)

The solar engine can handle these joint models as well; load datasets are added to the engine in the process of fitting.

In [ ]:
from pgscen.engine import SolarGeminiEngine

se = SolarGeminiEngine(solar_site_actual_hists, solar_site_forecast_hists,
                       scen_start_time, solar_meta_df)
se.fit_load_solar_joint_model(load_zone_actual_hists,
                              load_zone_forecast_hists)

As with the solar-only case, we create a solar model for the daytime as well as the dawn and dusk hours, but now we also create a load model and a joint model for the day time period. In the joint model we aggregate solar sites according to the load zone they are in.

In [ ]:
import numpy as np
import seaborn as sns
from pgscen.utils.plot_utils import get_clustermat, cov_cmap
from itertools import combinations as combn
import networkx as nx


def draw_pcor_graph(pcor_mat, asset_list, ax, plt_thresh, **nxargs):
    asset_graph = nx.Graph()

    for (i, asset1), (j, asset2) in combn(enumerate(asset_list), 2):
        if np.abs(pcor_mat[i, j]) >= plt_thresh:
            asset_graph.add_edge(asset1, asset2, pcor=pcor_mat[i, j])
    
    nx.draw(asset_graph, ax=ax, with_labels=True, 
            node_color='green', font_weight='bold',
            edge_color=[(0.10, 0.37, 1., 0.47) if val > 0 else (1., 0., 0., 0.47)
                        for val in nx.get_edge_attributes(asset_graph, 'pcor').values()],
            width=[53 * (val - plt_thresh)
                   for val in nx.get_edge_attributes(asset_graph, 'pcor').values()],
            **nxargs)


fig, (load_ax, solar_ax, joint_ax) = plt.subplots(
    figsize=(15, 23), nrows=3, ncols=1)

load_cov = se.gemini_dict['day']['load_model'].asset_cov
load_prec = np.linalg.inv(load_cov)
load_pcor = -load_prec / np.sqrt(np.outer(np.diag(load_prec),
                                          np.diag(load_prec)))
draw_pcor_graph(load_pcor, load_cov.index, load_ax, 0.1, font_size=17)


solar_cov = se.gemini_dict['day']['solar_model'].asset_cov
D = np.sqrt(np.diag(solar_cov))
corr_mat = (D ** -1) * solar_cov * (D ** -1)

solar_clust = get_clustermat(corr_mat)
sns.heatmap(solar_clust, ax=solar_ax, cmap=cov_cmap,
            vmin=-1, vmax=1, square=True)


joint_cov = se.gemini_dict['day']['joint_model'].asset_cov
joint_prec = np.linalg.inv(joint_cov)
joint_pcor = -joint_prec / np.sqrt(np.outer(np.diag(joint_prec),
                                            np.diag(joint_prec)))
draw_pcor_graph(joint_pcor, joint_cov.index, joint_ax, 0.15, font_size=13)

    
fig.tight_layout()    

Note that for scenario generation we must show the models both the future solar generator output and the future load demands.

In [ ]:
se.create_load_solar_joint_scenario(scenario_count,
                                    load_zone_forecast_futures,
                                    solar_site_forecast_futures)

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
from ipywidgets import interact, Dropdown
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


gdf = gpd.GeoDataFrame(
    se.meta_df, geometry=[Point(xy) for xy in zip(se.meta_df['longitude'],
                                                  se.meta_df['latitude'])]
    )

tx = gpd.read_file(Path(data_dir, "MetaData", "Texas_State_Boundary-shp.zip"))


def plot_solar_scenarios(solar_site):
    ax = tx.plot(facecolor="none", edgecolor='black', figsize=(18, 18), linewidth=2.3)
    gdf.plot(ax=ax, marker='o', linewidth=0, markersize=71, c='green')

    cntr_x, cntr_y = se.meta_df.longitude.mean(), se.meta_df.latitude.mean()
    site_x, site_y = se.meta_df.loc[solar_site, ['longitude', 'latitude']]
    xdiff, ydiff = site_x - cntr_x, site_y - cntr_y

    cntr_dist = (xdiff ** 2 + ydiff ** 2) ** 0.5
    site_cos, site_sin = xdiff / cntr_dist, ydiff / cntr_dist
    scen_x, scen_y = cntr_x + 8.5 * site_cos, cntr_y + 8.5 * site_sin

    scen_size = 7.5, 4
    scen_bbox = (scen_x - scen_size[0] / 2, scen_y - scen_size[1] / 2,
                 scen_size[0], scen_size[1])

    scen_ax = inset_axes(ax, width='100%', height='100%',
                         bbox_to_anchor=scen_bbox, bbox_transform=ax.transData,
                         loc=10, borderpad=0)
    
    ax.plot([site_x + 0 * site_cos, scen_x - 3.8 * site_cos],
            [site_y + 0 * site_sin, scen_y - 2 * site_sin],
            linewidth=4.3, c='black', alpha=0.53, linestyle='dotted')
    
    for i in range(scenario_count):
        scen_ax.plot(se.scenarios['solar'].iloc[i][solar_site],
                     c='black', alpha=0.13, lw=0.2)

    plt_fcst = se.forecasts['solar'][solar_site]
    scen_ax.plot(plt_fcst, c=fcst_clr, alpha=0.47, lw=4.1)
    scen_ax.plot(solar_site_actual_futures.loc[plt_fcst.index, solar_site],
                 c=actual_clr, alpha=0.47, lw=4.1)

    scen_ax.xaxis.set_major_formatter(mdates.DateFormatter("%H:%M"))
    scen_ax.tick_params(axis='both', labelsize=17)
    scen_ax.grid(linewidth=0.9, alpha=0.41)
    scen_ax.set_ylim((0, scen_ax.get_ylim()[1]))

    quant_df = se.scenarios['solar'][solar_site].quantile([0.25, 0.75])
    scen_ax.fill_between(quant_df.columns, quant_df.iloc[0], quant_df.iloc[1],
                         color='red', alpha=0.31)

    lgnd_ptchs = [Patch(color='black', alpha=0.23, label="Scenarios"),
                  Patch(color='red', alpha=0.41, label="Interquartile Range"),
                  Patch(color=fcst_clr, alpha=0.81, label="Forecast"),
                  Patch(color=actual_clr, alpha=0.81, label="Actual")]

    _ = scen_ax.legend(handles=lgnd_ptchs, frameon=False,
                       fontsize=11, ncol=2, handletextpad=0.5)

    ax.axis('off')

    
w = Dropdown(options=se.asset_list, description="Scenarios for generator:",
             layout={'align_self': 'center'}, style={'description_width': 'initial'},
             disabled=False)
_ = interact(plot_solar_scenarios, solar_site=w)

In [ ]:
def plot_zone_scenarios(plt_zone):
    fig, ax = plt.subplots(figsize=(15, 7))

    for i in range(scenario_count):
        plt.plot(se.scenarios['load'].iloc[i][plt_zone],
                 c='black', alpha=0.13, lw=0.2)

    plt_fcst = se.forecasts['load'][plt_zone]
    plt.plot(plt_fcst, c=fcst_clr, alpha=0.47, lw=4.1)
    plt.plot(load_zone_actual_futures.loc[plt_fcst.index, plt_zone],
             c=actual_clr, alpha=0.47, lw=4.1)

    quant_df = se.scenarios['load'][plt_zone].quantile([0.25, 0.75])
    ax.fill_between(quant_df.columns, quant_df.iloc[0], quant_df.iloc[1],
                    color='red', alpha=0.31)

    ax.tick_params(which='both', labelsize=13)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%m/%d'))
    ax.grid(linewidth=0.9, alpha=0.41)
    ax.axhline(0, lw=1.3, c='black', alpha=1)
    ax.set_ylim((0, ax.get_ylim()[1]))
    
    lgnd_ptchs = [Patch(color='black', alpha=0.23, label="Scenarios"),
                  Patch(color='red', alpha=0.41, label="Interquartile Range"),
                  Patch(color=fcst_clr, alpha=0.81, label="Forecast"),
                  Patch(color=actual_clr, alpha=0.81, label="Actual")]

    _ = plt.legend(handles=lgnd_ptchs, frameon=False, fontsize=17, ncol=4, loc=8,
                   bbox_to_anchor=(0.5, -0.17), handletextpad=0.7)

w = Dropdown(options=load_zone_actual_df.columns, description="Scenarios for zone:",
             layout={'align_self': 'center'}, style={'description_width': 'initial'},
             disabled=False)
_ = interact(plot_zone_scenarios, plt_zone=w)